# LEAFToolbox for Sites
Applies the LEAF-ToolBox for mapping vegetation using satellite imagery over a list of sites, each defined using time stamped vector geographical objects.
Cite as Fernandes, R. et al., 2021, "LEAF Toolbox", Canada Centre for Remote Sensing, https://github.com/rfernand387/LEAF-Toolbox/wiki, DOI: 10.5281/zenodo.4321298.


LEAF currently supports two algorithms: 

SL2PV0 is the algorithm defined by Weiss and Baret (2016) with an update in Weiss and Baret (2020).  


SL2PV1.1 defined in  Fernandes et al. 2024 that attempts to correct for biases in crown and shoot clumping over forests observed in SL2PV0.


Refer to https://github.com/rfernand387/LEAF-Toolbox/tree/master/Source-Python for configuration of anaconda environment.


Weiss, M. and Baret, F. 2016. S2ToolBox Level 2 products: LAI, FAPAR, FCOVER, 1.1. ed.
Institut National de la Recherche Agronomique, Avignon, France. https://step.esa.
int/docs/extra/ATBD_S2ToolBox_L2B_V1.1.pdf.

Weiss, M., and Baret, F., 2020. S2ToolBox Level 2 Products: LAI, FAPAR, FCOVER, 2.0. ed.
Institut National de la Recherche Agronomique, Avignon, France. https://step.esa.
int/docs/extra/ATBD_S2ToolBox_L2B_V2.0.pdf.

Fernandes, R., Canisius, F., Harvey, K., Hong, G., MacDougall, C., Shah, H., Sun, L., 2024. Evidence of a bias-variance trade off when correcting for bias in Sentinel 2 forest LAI retrievals using radiative transfer models. Remote Sensing of EnvironmentRemote Sens. Environ. 305. doi:10.1016/j.rse.2024.114060.


## Setup Environment

In [2]:
# Provide your GEE authentificaton
import ee
ee.Authenticate()

Enter verification code:  4/1ATX87lNdlTnu7_3QC14rClvDuRXKOVBJgqp69xEgNjR5H_maQE3TsWP-JVM



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [ ]:
# Uncomment this if you are modifying modules
# %load_ext autoreload
# %autoreload 2

In [6]:
# import LEAF modules
import LEAFv3

# # import algorithm definitions
import SL2PV0 
import SL2PV1
import SL2PV1MIX
import SL2PV1DBF
import SL2PV1ENF
import SL2PV10MIX
import SL2PV10DBF
import SL2PV10ENF

In [9]:
import LEAFv4

In [ ]:
pip install geetools

^C
Note: you may need to restart the kernel to use updated packages.


## Process Sites with output as a Pandas Data Frame

Sites correspond to readable Google Earth Engine Feature collections under which samples will be extracted.
If the features have time stamps as properties the sampling interval can be restricted on a feature by feature basis.
Otherwise the sampling interval must be specified the same for all features.

In [80]:
# List of GEE feature collections
# siteList = ["projects/ee-modis250/assets/sl2pclass100kmveg20mbuf"]
# siteList=["projects/ee-modis250/assets/FO_HWSW_Buf60_lonlat"]
# siteList=["projects/ee-modis250/assets/northern_filtered_abandoned_wells","projects/ee-modis250/assets/northern_reference_buffers_matched"]
# siteList=["projects/ee-modis250/assets/sample_wells","projects/ee-modis250/assets/sample_wells_reference_buffers"]
siteList=['projects/ee-modis250/assets/2bt/Bonney_SiteswithDates_2019']
# siteList=["projects/ee-modis250/assets/sites_3013_buffer150m"]
# siteList=["projects/ee-ronnyale/assets/random_sample_1000_filtered_abandoned_wells"]
# siteList=["projects/ee-modis250/assets/Anderson/Anderson_LAI-2017-2018_May16to24","projects/ee-modis250/assets/Anderson/Anderson_LAI-2017-2018_Aug16to24"]

## Run LEAF 

In [14]:
# Function definition
#
# LEAF.sampleSites
# def sampleSites(siteList,
#                 imageCollectionName,
#                 sensorName,
#                 outputPath,
#                 algorithm,
#                 variableName='Surface_Reflectance',
#                 featureRange=[0,np.nan],
#                 maxCloudcover=100,
#                 outputScale=30,
#                 exportFlag='Asset',
#                 prefixProperty='system:index',
#                 inputScale=30,
#                 bufferSpatialSize=0,
#                 subsamplingFraction=1,
#                 numPixels=0,
#                 geometries=False,
#                 bufferTemporalSize=[0,0],
#                 calendarRange=[1,12,'month'],
#                 timeZone='Etc/GMT+6',
#                 shardSize='ys'
#                 )
#
# Applies a LEAF toolbox algorithm to map a canopy variable for all clear sky unmasked pixels from an input image collection
# falling within the spatial and temporal extents of features in a list of sites.  The spatial extent of features is defined by their GEE geometry.  
# The temporal extent of features is defined by their "system:time_start" and "system:time_end" properties or user defined otherwise.
#
# Parameters:
#
# siteList: list of readable GEE feature collection assets.  Features must have a system:time_start and system:time_end property.
# sensorName: user provided name of sensor for output file name
# imageCollectionName: input image collection name from one of the list defined by GEE
#                 ["COPERNICUS/S2_SR_HARMONIZED","COPERNICUS/S2_SR_HARMONIZED_10m","LANDSAT/LC08/C02/T1_L2","LANDSAT/LC09/C02/T1_L2","NASA/HLS/HLSL30/v002"]
# outputPath: google drive , cloud or gee asset folder (for cloud or asset it must exist)
# algorithm: python module specifying LEAF algorithm to apply , currently one of list ["SL2PV0","SL2PV1"]
# variableName: variable to be mapped from one of the list ["ALBEDO","FAPAR","FCOVER","LAI","CWC","CCC","DASF"] defined by https://github.com/rfernand387/LEAF-Toolbox/wiki/Visualisation-Outputs              
# feature_range: range of features in each site GEE feature collection to sample, use this if you have one site with many features and want only a few of them                                                                                                                        
# maxCloudCover: maximum input image cloud cover percentage [0,100]
# outputScale: output pixel size (m)
# exportFlag: export destination (Local,Drive,Asset,CloudStorage) , dont use Local unless it is a small job
# prefixProperty: the property of the sampling features to use for file name prefix
# inputScale: input pixel size (m)
# bufferSpatialSize: width (m) of spatial buffer applied to features prior to sampling
# bufferTemporalSize: dilation (d) of start and end time of sampled features OR temporal interval ["YYYY-MM-DD","YYYY-MM-DD"[ of start and end of time interval to sample.  
# subsamplingFraction: fraction (0 to 1) of pixels to sample within a feature on any given image
# numPixels: maximum number of pixels to sample within a feature, overrides subsampling fraction
# geometries: export the geometry of the sampling feature
# calendarRange: months within each year to sample, inclusive
# timeZone: time zone for sampling if desired, otherwise UTC
# shardSize: temporal interval of shards for output files, e.g. annual
#
# Output:
#
# Drive,Asset,CloudStorage: .csv file with one row per sampled output pixel per day
# Local: Results in a .pkl file with a pandas dictionary for each site in siteList:
#
# sitesDictionary: dictionary with keys corresponding to sites and structure.  The value of each key is a dictionary with two keys:
#                     'feature' : properties of the sampled feature
#                     'SL2PV0' or 'SL2PV1' : a pandas data frame where columns correspond to algorithm output properties and rows correspond to a retrieval within the time and spatial interval of the feature               
#
# Example:
#S2LAIV0_2025= LEAFv4.sampleSites(siteList, 
# sensorName = 'S2', 
# imageCollectionName="COPERNICUS/S2_SR_HARMONIZED",
# outputPath='leafoutput13',
# algorithm=SL2PV0, 
# variableName="LAI",
# featureRange=[0,999], 
# maxCloudcover=20,
# outputScale=20,
# exportFlag='Drive',
# prefixProperty='system:index',
# inputScale=20,
# bufferSpatialSize=0,
# subsamplingFraction=1,
# numPixels=0,
# geometries=False,
# bufferTemporalSize=['2017-04-01','2025-12-31'],
# calendarRange=[6,9,'month'],
# timeZone='Etc/GMT+6',
# shardSize='YS')
#

In [ ]:
S2LAIV0_2025= LEAFv4.sampleSites(siteList, 
                                    sensorName = 'S2', 
                                    imageCollectionName="COPERNICUS/S2_SR_HARMONIZED",
                                    outputPath='leafoutput13',
                                    algorithm=SL2PV0, \
                                    variableName="LAI",
                                    featureRange=[0,999], 
                                    maxCloudcover=20,
                                    outputScale=20,
                                    exportFlag='Drive',
                                    prefixProperty='system:index',
                                    inputScale=20,
                                    bufferSpatialSize=0,
                                    subsamplingFraction=1,
                                    numPixels=0,
                                    geometries=False,
                                    bufferTemporalSize=['2017-04-01','2025-12-31'],
                                    calendarRange=[6,9,'month'],
                                    timeZone='Etc/GMT+6',
                                    shardSize='YS')


STARTING LEAF.sampleSites using  COPERNICUS/S2_SR_HARMONIZED
Site:  projects/ee-modis250/assets/2bt/Bonney_SiteswithDates_2019  with  27  features.
Processing Features: from 0 to 27
Feature n°: 0/27  -- startDate: 2017-04-01 00:00:00 -- endDate: 2025-12-31 00:00:00
----------------------------------------------------------------------------------------------------------
Sampling from  2017-04-01 00:00:00  to  2018-01-01 00:00:00
['date', 'QC', 'longitude', 'latitude', 's2cloudless_probability', 'estimateLAI', 'partition', 'networkID', 'errorLAI']
Export task started with ID: 3C6FL5VAT7H47VOPPX7V62CI
Sampling from  2018-01-01 00:00:00  to  2019-01-01 00:00:00
['date', 'QC', 'longitude', 'latitude', 's2cloudless_probability', 'estimateLAI', 'partition', 'networkID', 'errorLAI']
Export task started with ID: LBK3KPIXXKRWJDYFXX2O62QL
Sampling from  2019-01-01 00:00:00  to  2020-01-01 00:00:00
['date', 'QC', 'longitude', 'latitude', 's2cloudless_probability', 'estimateLAI', 'partition', 'net

In [ ]:
# Function definition
#
# LEAF.imageSites
# def imageSites(siteList,
#                 imageCollectionName,
#                 sensorName,
#                 outputPath,
#                 algorithm,
#                 variableName='Surface_Reflectance',
#                 featureRange=[0,np.nan],
#                 maxCloudcover=100,
#                 outputScale=30,
#                 exportFlag='Asset',
#                 prefixProperty='system:index',
#                 inputScale=30,
#                 bufferSpatialSize=0,
#                 subsamplingFraction=1,
#                 numPixels=0,
#                 bufferTemporalSize=[0,0],
#                 calendarRange=[1,12,'month'],
#                 timeZone='Etc/GMT+6',
#                 shardSize='ys',
#                 exportDataType='float'
#                 ):
#
# Applies a LEAF toolbox algorithm to map a canopy variable for all clear sky unmasked pixels from an input image collection
# falling within the spatial and temporal extents of features in a list of sites.  The spatial extent of features is defined by their GEE geometry.  
# The temporal extent of features is defined by their "system:time_start" and "system:time_end" properties or user defined otherwise.
#
# Parameters:
#
# siteList: list of readable GEE feature collection assets.  Features must have a system:time_start and system:time_end property.
# sensorName: user provided name of sensor for output file name
# imageCollectionName: input image collection name from one of the list defined by GEE
#                 ["COPERNICUS/S2_SR_HARMONIZED","COPERNICUS/S2_SR_HARMONIZED_10m","LANDSAT/LC08/C02/T1_L2","LANDSAT/LC09/C02/T1_L2","NASA/HLS/HLSL30/v002"]
# outputPath: google drive , cloud or gee asset folder (for cloud or asset it must exist)
# algorithm: python module specifying LEAF algorithm to apply , currently one of list ["SL2PV0","SL2PV1"]
# variableName: variable to be mapped from one of the list ["ALBEDO","FAPAR","FCOVER","LAI","CWC","CCC","DASF"] defined by https://github.com/rfernand387/LEAF-Toolbox/wiki/Visualisation-Outputs              
# feature_range: range of features in each site GEE feature collection to sample, use this if you have one site with many features and want only a few of them                                                                                                                        
# maxCloudCover: maximum input image cloud cover percentage [0,100]
# outputScale: output pixel size (m)
# exportFlag: export destination (Local,Drive,Asset,CloudStorage) , dont use Local unless it is a small job
# prefixProperty: the property of the sampling features to use for file name prefix
# inputScale: input pixel size (m)
# bufferSpatialSize: width (m) of spatial buffer applied to features prior to sampling
# bufferTemporalSize: dilation (d) of start and end time of sampled features OR temporal interval ["YYYY-MM-DD","YYYY-MM-DD"[ of start and end of time interval to sample.  
# subsamplingFraction: fraction (0 to 1) of pixels to sample within a feature on any given image
# numPixels: maximum number of pixels to sample within a feature, overrides subsampling fraction
# geometries: export the geometry of the sampling feature
# calendarRange: months within each year to sample, inclusive
# timeZone: time zone for sampling if desired, otherwise UTC
# shardSize: temporal interval of shards for output files, e.g. annual
#
# Output:
#
# Drive,Asset,CloudStorage: .csv file with one row per sampled output pixel per day
# Local: Results in a .pkl file with a pandas dictionary for each site in siteList:
#
# sitesDictionary: dictionary with keys corresponding to sites and structure.  The value of each key is a dictionary with two keys:
#                     'feature' : properties of the sampled feature
#                     'SL2PV0' or 'SL2PV1' : a pandas data frame where columns correspond to algorithm output properties and rows correspond to a retrieval within the time and spatial interval of the feature               
#
# Example:
#S2LAIV0_2025= LEAFv4.sampleSites(siteList, 
# sensorName = 'S2', 
# imageCollectionName="COPERNICUS/S2_SR_HARMONIZED",
# outputPath='leafoutput13',
# algorithm=SL2PV0, 
# variableName="LAI",
# featureRange=[0,999], 
# maxCloudcover=20,
# outputScale=20,
# exportFlag='Drive',
# prefixProperty='system:index',
# inputScale=20,
# bufferSpatialSize=0,
# subsamplingFraction=1,
# numPixels=0,
# geometries=False,
# bufferTemporalSize=['2017-04-01','2025-12-31'],
# calendarRange=[6,9,'month'],
# timeZone='Etc/GMT+6',
# shardSize='YS')
#

In [ ]:
HLSL30LAIV0_2025= LEAFv4.imageSites(siteList, \
                                    imageCollectionName="COPERNICUS/S2_SR_HARMONIZED",\
                                    sensorName = 'S2', \
                                    outputPath='leafoutput8',\
                                    algorithm=SL2PV0, \
                                    variableName="LAI",\
                                    featureRange=[0,1], \
                                    maxCloudcover=80,\
                                    outputScale=20,\
                                    exportFlag='CloudStorage',
                                    prefixProperty='system:index',
                                    inputScale=20,
                                    bufferSpatialSize=4000,
                                    bufferTemporalSize=['2020-07-01','2021-09-01'],
                                    calendarRange=[7,8,'month'],
                                    timeZone='Etc/GMT+6',
                                    shardSize='YS')

STARTING LEAF.imageSites using  COPERNICUS/S2_SR_HARMONIZED
S2
Site:  projects/ee-modis250/assets/2bt/Bonney_SiteswithDates_2019  with  27  features.
Processing Features: from 0 to 1
Feature n°: 0/1  -- startDate: 2020-07-01 00:00:00 -- endDate: 2021-09-01 00:00:00
----------------------------------------------------------------------------------------------------------
Sampling from  2020-07-01 00:00:00  to  2021-09-01 00:00:00
Export task started with ID: AGXLKEP5L7DR6XAYMNJBEKES
Export task started with ID: MVJIZUIXFSIW7B4WBH54HEOP
Export task started with ID: JRSP6OZCAMNRGZA7KESU5DFC
Export task started with ID: WOBX6Q4A3JLMECCBPKNAAJBU
Export task started with ID: 2NRJVRUIM2WBWFC42IN7QAYZ
Export task started with ID: 735HTJT3NRXUIV6XADOL3SS5
Export task started with ID: CHXWSZW5CZRBUW6FGT2ZUMOM
Export task started with ID: XLSVL3ZTETL6RMYRB7C7PGMD
Export task started with ID: VFRR2KQD5LSVAG56QZNUG6TN
Export task started with ID: M3BQXMG5QYC6LW6IFYSQZ32P
Export task started with ID